In [1]:
#本程序用单纯形法求解线性规划问题，在求解之前要求问题已经转化为标准问题，使用两阶段法求解的问题需要先写为辅助问题求解一次后，人工定义新的问题再次求解
#单纯形法是迭代算法，在每一轮迭代中，本程序都将输出完整的系数矩阵、右侧向量、判别式、基变量指标集和非基变量指标集
#请特别注意，中间过程的指标集都是有序的，因此右侧向量和判别式各分量分别与基变量指标集和非基变量指标集一一对应，单纯形表遵循以下格式：
#假设基变量为[2,5]，非基变量指标集[1,4,6,3]，记判别式为d，系数矩阵为a，右侧向量为b
#   x1 | x2 | x3 | x4 | x5 | x6
#   d1 | 0  | d4 | d2 | 0  | d3
# 2 a11| a12| a13| a14| a15| a16 | b1
# 5 a21| a22| a23| a24| a25| a26 | b2
#但最终输出的A, b, x都是按照正常顺序（指标从小到大）排列，这是为了方便两阶段法的计算，确保A,b与原问题的c顺序对应
#完整算法可参考 最优化方法/杨庆之编著.-北京:科学出版社,2015, 算法2.5.5 修正单纯形法
import numpy as np
import simplex_method as sm

#第一阶段: 求解辅助问题

c = np.array([0,0,0,0,1,1])

A = np.array([
    [1, 3, 4, -1, 1, 0], 
    [2, 1, 0,  0, 0, 1]
])

b = np.array([10, 12])

IB = [4, 5]

A ,b, x, IB = sm.simplex_method(A, b, c, IB)

当前第1轮,基变量指标集: [5, 6], 非基变量指标集: [1, 2, 3, 4]
系数矩阵为: [[ 1  3  4 -1  1  0]
 [ 2  1  0  0  0  1]]
右侧向量为: [10 12]
判别式为: [-3. -4. -4.  1.]
进基变量指标: 2 出基变量指标: 5
当前第2轮,基变量指标集: [2, 6], 非基变量指标集: [1, 5, 3, 4]
系数矩阵为: [[ 0.33333333  1.          1.33333333 -0.33333333  0.33333333  0.        ]
 [ 1.66666667  0.         -1.33333333  0.33333333 -0.33333333  1.        ]]
右侧向量为: [3.33333333 8.66666667]
判别式为: [-1.66666667  1.33333333  1.33333333 -0.33333333]
进基变量指标: 1 出基变量指标: 6
当前第3轮,基变量指标集: [2, 1], 非基变量指标集: [6, 5, 3, 4]
系数矩阵为: [[ 0.   1.   1.6 -0.4  0.4 -0.2]
 [ 1.   0.  -0.8  0.2 -0.2  0.6]]
右侧向量为: [1.6 5.2]
判别式为: [1. 1. 0. 0.]
问题有解,基变量指标集为: [1, 2]
最优解为: [5.2 1.6 0.  0.  0.  0. ]


In [2]:
# 第二阶段: 求解原问题
A = A[:, :4]
c = np.array([24, 48, 36, 0])
A ,b, x, IB = sm.simplex_method(A, b, c, IB)

当前第1轮,基变量指标集: [1, 2], 非基变量指标集: [3, 4]
系数矩阵为: [[ 1.   0.  -0.8  0.2]
 [ 0.   1.   1.6 -0.4]]
右侧向量为: [5.2 1.6]
判别式为: [-21.6  14.4]
进基变量指标: 3 出基变量指标: 2
当前第2轮,基变量指标集: [1, 3], 非基变量指标集: [2, 4]
系数矩阵为: [[ 1.     0.5    0.     0.   ]
 [ 0.     0.625  1.    -0.25 ]]
右侧向量为: [6. 1.]
判别式为: [13.5  9. ]
问题有解,基变量指标集为: [1, 3]
最优解为: [6. 0. 1. 0.]


In [3]:
# 第三阶段：用scipy的线性规划求解器检验结果
from scipy.optimize import linprog

c = np.array([24, 48, 36])

A_ub = np.array([
    [-1, -3, -4],
    [-2, -1,  0]
])
b_ub = np.array([-10, -12])

A_eq = np.array([
    [2, 1, 0]
])
b_eq = np.array([12])

bounds = [(0, None), (0, None), (0, None)]

result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)

print("最优解:", result.x)
print("最优目标值:", result.fun)
print("求解状态:", result.message)

最优解: [6. 0. 1.]
最优目标值: 180.0
求解状态: Optimization terminated successfully. (HiGHS Status 7: Optimal)
